https://www.crummy.com/software/BeautifulSoup/bs4/doc/

http://docs.python-requests.org/en/master/user/quickstart/

https://www.v2ex.com/

In [1]:
from __future__ import print_function    # (at top of module)
from __future__ import unicode_literals

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import time


page_to_crawl = 1 # change this to crawl more
items = list()
base_url = 'https://www.v2ex.com'
is_banned = False

for npage in range(1, page_to_crawl+1):
    r = requests.get('https://www.v2ex.com/go/programmer?p=' + str(npage))
    soup = BeautifulSoup(r.text, 'lxml')

    for item in soup.find_all(class_='item_title'):
        title = item.text
        item_url = base_url + item.a.get('href')
        print(item_url, end=',')
        
        _r = requests.get(item_url)
        print(_r.status_code)
        if _r.status_code == 403: # banned
            is_banned = True
            break
            
        _soup = BeautifulSoup(_r.text, 'lxml')

        tags = list(map(lambda tag: tag.text, _soup.find_all(class_='tag')))

        cont = list(map(lambda x: x.text, soup.find_all(class_='topic_content')))
        reply = list(map(lambda x: x.text, soup.find_all(class_='reply_content')))
        content = u'。'.join(cont+reply)
        
        item_in_json = {'title': title, 
                        'tags': tags,
                        'content': content
                       }
        
        items.append(item_in_json)
        
        time.sleep(3) # delays for seconds
    
    if is_banned:
        break

https://www.v2ex.com/t/350701#reply2,200
https://www.v2ex.com/t/350696#reply1,200
https://www.v2ex.com/t/350464#reply49,200
https://www.v2ex.com/t/350602#reply14,200
https://www.v2ex.com/t/350538#reply101,200
https://www.v2ex.com/t/350522#reply76,200
https://www.v2ex.com/t/350535#reply36,200
https://www.v2ex.com/t/350697#reply0,200
https://www.v2ex.com/t/350456#reply55,200
https://www.v2ex.com/t/350634#reply8,200
https://www.v2ex.com/t/350462#reply21,200
https://www.v2ex.com/t/350503#reply18,200
https://www.v2ex.com/t/350524#reply4,200
https://www.v2ex.com/t/350606#reply0,200
https://www.v2ex.com/t/350551#reply5,200
https://www.v2ex.com/t/350605#reply3,200
https://www.v2ex.com/t/350623#reply0,200
https://www.v2ex.com/t/350489#reply15,200
https://www.v2ex.com/t/350685#reply3,200
https://www.v2ex.com/t/350410#reply40,200


In [5]:
print(len(items))
with open('v2ex.json', 'w') as fout:
    fout.write(json.dumps(items))

0


In [11]:
with open('v2ex.json', 'r') as fin:
    v2ex = json.loads(fin.read())

In [12]:
import re

from text_cleaner import remove, keep
from text_cleaner.processor.common import SYMBOLS_AND_PUNCTUATION_EXTENSION
from text_cleaner.processor.chinese import CHINESE_SYMBOLS_AND_PUNCTUATION
from text_cleaner.processor.misc import RESTRICT_URL


docs = map(lambda x: u'{0} {1}'.format(x['title'], x['content']), v2ex)
corpus = u' '.join(docs)
corpus = remove(corpus, 
    [RESTRICT_URL, CHINESE_SYMBOLS_AND_PUNCTUATION])

print(len(corpus))

18316


In [13]:
max_word_size = 7
min_word_size = 2
margin_size = 1

from collections import defaultdict
freq = defaultdict(int)
w2id = dict()
id2w = list()
left = defaultdict(list)
right = defaultdict(list)

for chunk in corpus.split():
    for sz in range(1, max_word_size+1):
        for beg in range(0, sz):
            for idx in range(beg, len(chunk), sz):
                w = chunk[idx:idx+sz]
                l = chunk[idx-margin_size:idx] if idx>=margin_size else ''
                r = chunk[idx+sz:idx+sz+margin_size]

                if w not in w2id:
                    w2id[w] = len(w2id)+1
                freq[w] = freq[w] + 1
                
                if l:
                    if l not in w2id:
                        w2id[l] = len(w2id)+1
                    left[w].append(w2id[l])
                if r:
                    if r not in w2id:
                        w2id[r] = len(w2id)+1            
                    right[w].append(w2id[r])

In [14]:
import operator

# id2w is for debugging use, optional
id2w.append('^') # not 0-indexed
for w,i in sorted(w2id.items(), key=operator.itemgetter(1), reverse=False):
    id2w.append(w)

In [58]:
import sys
import math
from collections import Counter

N = float(len(corpus))

freqs = list(
    map(operator.itemgetter(1), 
        filter(lambda x: len(x[0])>=min_word_size, 
               freq.items())))
average_freq = sum(freqs) / len(freqs)

inner_closeness = defaultdict(float)
for w,cnt in freq.items():
    if len(w) < min_word_size:
        continue
        
    # only considering words composed of a pair of words
    denominator = 1
    for i in range(1, len(w)):
        l = w[:i]
        r = w[i:]
        _tmp = freq[l] * freq[r]
        denominator = _tmp if _tmp>denominator else denominator
    
    entropy = lambda p: -p * math.log(p)
    
    l = left[w]
    lfreq = Counter(l)
    if freq[w] - len(l) > 0:
        lfreq[-1] = freq[w] - len(l) # edge cases
    left_closeness = sum(map(entropy, map(lambda n: n/freq[w], lfreq.values())))
    
    r = right[w]
    rfreq = Counter(r)
    if freq[w] - len(r) > 0:
        rfreq[-1] = freq[w] - len(r)
    right_closeness = sum(map(lambda n: entropy(n/freq[w]), rfreq.values()))
    
    # p(w1+w2)/ (p(w1)*p(w2))
    inner = (freq[w]) / (denominator+average_freq) * N
    
#     inner_closeness[w] = freq[w] * N / denominator
#     inner_closeness[w] = min(left_closeness, right_closeness) + math.log(freq[w] * N / denominator)
    inner_closeness[w] = min(left_closeness, right_closeness) + math.log(inner)

In [59]:
# v1.4: freq[w]+average, min(left,right) + inner closeness
for w,prob in sorted(inner_closeness.items(), key=operator.itemgetter(1), reverse=True)[:100]:
    print(w, prob)

吐槽 9.605649187090073
靠谱 9.605649187090073
淘宝 9.401910987089408
轨迹 9.398819490264376
伪娘 9.398819490264376
揭秘 9.398819490264376
仲裁 9.398819490264376
描述 9.398819490264376
挑战 9.398819490264376
窥豹 9.398819490264376
八折 9.398819490264376
陈斌 9.398819490264376
约炮 9.398819490264376
竞猜 9.398819490264376
允许 9.398819490264376
判断 9.393243724197784
痛苦 9.29457786825095
铁克 9.29457786825095
斐迅 9.29457786825095
抢购 9.29457786825095
拒绝 9.29457786825095
闪退 9.29457786825095
侯延堂 9.216497933470421
爱奇艺 9.216497933470421
烂大街 9.216497933470421
夸张 9.20018407898191
厉害 9.20018407898191
爆洞 9.20018407898191
捂脸 9.20018407898191
游戏 9.173572164299182
世界 9.143551066716777
货满满 9.112256311456996
再迷茫 9.112256311456996
覆盖 9.109740558978938
挖掘 9.109740558978938
拧花 9.034541467747134
仓详细 9.017862522187954
杂度低 9.017862522187954
兴趣 9.011946367770365
贝的烦恼 8.993354382156213
革命 8.960988548545531
福利 8.960988548545531
拇指按 8.931615445992929
华硕 8.909553141330461
迷茫 8.909553141330461
演示 8.892476972282058
严重 8.892476972282058
贼鸡巴慢 8.889112

In [42]:
# v1.3: min(left,right) + inner closeness
for w,prob in sorted(inner_closeness.items(), key=operator.itemgetter(1), reverse=True)[:100]:
    print(w, prob)

糊其词的扯 9.815530273776986
尼玛 9.815530273776986
堪称神兵 9.815530273776986
浪股价大跌 9.815530273776986
贡献 9.815530273776986
蜡烛 9.815530273776986
彩的二哲 9.815530273776986
风清 9.815530273776986
堪称 9.815530273776986
潦草 9.815530273776986
乍现”的那些瞬 9.815530273776986
械革 9.815530273776986
飞蚊症严 9.815530273776986
吐槽 9.815530273776986
锐速的官 9.815530273776986
毫无色彩的二哲 9.815530273776986
巡礼 9.815530273776986
蚂蚁 9.815530273776986
玛是复制粘 9.815530273776986
负责 9.815530273776986
飞蚊症 9.815530273776986
光乍 9.815530273776986
阮一峰 9.815530273776986
曾经一起北漂 9.815530273776986
坛上有童 9.815530273776986
螺丝总是被我拧 9.815530273776986
均衡 9.815530273776986
称神兵 9.815530273776986
顺便有偿 9.815530273776986
粘贴 9.815530273776986
住宿 9.815530273776986
鼠标产生的的轨 9.815530273776986
混淆 9.815530273776986
蓝桥 9.815530273776986
沈阳 9.815530273776986
左右滑 9.815530273776986
稀土 9.815530273776986
犹豫 9.815530273776986
凸显比较厉 9.815530273776986
靠谱 9.815530273776986
谷歌 9.815530273776986
童鞋 9.815530273776986
店铺 9.815530273776986
崇拜 9.815530273776986
照规律 9.815530273776986
飞蚊

In [38]:
# v1.2: min(left,right) closeness
for w,prob in sorted(inner_closeness.items(), key=operator.itemgetter(1), reverse=True)[:100]:
    print(w, prob)

怎么 2.435932088529907
什么 2.344224997654852
是不是 2.3383717048035724
如何 2.3094584236946436
可以 2.3056573382986394
自己 2.218701935467376
一个 2.1762880567264364
开源 2.1639556568820564
数据 2.1216758631948687
面试 2.110016553375461
服务 1.9919512045236958
li 1.9560115027140728
工作 1.9550839066643972
ro 1.9468389992053854
淘宝 1.945910149055313
有什么 1.935673269278717
用什么 1.9061547465398496
ri 1.896399636158174
比较 1.8946891171539901
re 1.8846299606459382
开发 1.870173695026971
程序员 1.8578069863704745
ti 1.8143075196071252
in 1.8135607453708442
服务器 1.7751250090009734
网络 1.771623955677366
as 1.7650572522256291
是如何 1.7478680974667573
免费 1.7478680974667573
是否 1.7478680974667573
pa 1.7367407460930633
lo 1.7361211828147631
觉得 1.7328679513998633
知道 1.7078211625602202
ma 1.7075391741350712
ec 1.6674619334292948
视频 1.6674619334292948
自动 1.666332585328325
一次 1.6313454208085967
用户 1.6216679171123507
状态 1.6094379124341005
没人 1.6094379124341005
北京 1.6094379124341005
这么 1.6094379124341005
oo 1.6094204431931665
大家 1.595953062

In [56]:
# v1.1: denominator==0 => sys.float_info.max, 
#+[RESTRICT_URL, SYMBOLS_AND_PUNCTUATION_EXTENSION, CHINESE_SYMBOLS_AND_PUNCTUATION]
for w,prob in sorted(inner_closeness.items(), key=operator.itemgetter(1), reverse=True)[:100]:
    print(w, prob)

糊其词的扯 18316.0
尼玛 18316.0
堪称神兵 18316.0
浪股价大跌 18316.0
贡献 18316.0
蜡烛 18316.0
彩的二哲 18316.0
风清 18316.0
堪称 18316.0
潦草 18316.0
乍现”的那些瞬 18316.0
械革 18316.0
飞蚊症严 18316.0
锐速的官 18316.0
毫无色彩的二哲 18316.0
巡礼 18316.0
蚂蚁 18316.0
玛是复制粘 18316.0
负责 18316.0
飞蚊症 18316.0
光乍 18316.0
阮一峰 18316.0
曾经一起北漂 18316.0
坛上有童 18316.0
螺丝总是被我拧 18316.0
均衡 18316.0
称神兵 18316.0
顺便有偿 18316.0
粘贴 18316.0
住宿 18316.0
鼠标产生的的轨 18316.0
混淆 18316.0
蓝桥 18316.0
沈阳 18316.0
左右滑 18316.0
稀土 18316.0
犹豫 18316.0
凸显比较厉 18316.0
谷歌 18316.0
童鞋 18316.0
店铺 18316.0
崇拜 18316.0
照规律 18316.0
飞蚊 18316.0
扇贝 18316.0
蚊症严 18316.0
玛是复制粘贴 18316.0
尼玛是复制粘贴 18316.0
哲和他的巡 18316.0
哲和他的巡礼 18316.0
另谋 18316.0
枫叶 18316.0
螺丝 18316.0
拍扁 18316.0
叶主机优惠 18316.0
措施 18316.0
枫叶主机优惠 18316.0
唤醒 18316.0
毫无色彩 18316.0
鲸鱼点亮生日蜡 18316.0
坛上有童鞋 18316.0
碑和评价差距 18316.0
吸引我的是画风 18316.0
尼玛是复制粘 18316.0
戴尔 18316.0
丝总是被我拧 18316.0
油猴 18316.0
症严 18316.0
杯竟 18316.0
蚊症 18316.0
轨迹 15699.42857142857
伪娘 15699.42857142857
揭秘 15699.42857142857
仲裁 15699.42857142857
描述 15699.42857142857
挑战 15699.42857142857


In [166]:
# v1.0
for w,prob in sorted(inner_closeness.items(), key=operator.itemgetter(1), reverse=True)[:100]:
    print(w, prob)

酷、腾讯、爱 17086.0
仲裁 17086.0
*768的分辨 17086.0
造sql语句 17086.0
症严 17086.0
缓存么？。伟 17086.0
蜡烛 17086.0
玛是复制粘 17086.0
巡礼 17086.0
油猴 17086.0
鲸鱼点亮生日蜡 17086.0
烂大街 17086.0
嘉，占 17086.0
贡献 17086.0
兰全球招募 17086.0
稀土 17086.0
锐速的官 17086.0
毫无色彩的二哲 17086.0
娘，还会被人崇 17086.0
鼠标产生的的轨 17086.0
八折 17086.0
吸引我的是画风 17086.0
蚂蚁 17086.0
败，各位大佬 17086.0
迹？。锐速的官 17086.0
曾经一起北漂 17086.0
爱奇艺 17086.0
均衡 17086.0
蚊症严 17086.0
描述 17086.0
货满满，更吸 17086.0
拍扁 17086.0
毫无色彩 17086.0
戴尔 17086.0
左右滑 17086.0
崇拜 17086.0
贝的烦恼 17086.0
累，想休 17086.0
顺便有偿 17086.0
瞬息。编码、摘 17086.0
喷。微信朋友圈 17086.0
哲和他的巡礼 17086.0
照规律 17086.0
枚，业余 17086.0
尼玛是复制粘贴 17086.0
乍现”的那些瞬 17086.0
枫叶 17086.0
哲和他的巡 17086.0
混淆 17086.0
童鞋 17086.0
揭秘 17086.0
伪娘 17086.0
风清 17086.0
飞蚊症严 17086.0
堪称神兵 17086.0
露，请尽 17086.0
住宿 17086.0
挑战 17086.0
丝总是被我拧 17086.0
尼玛 17086.0
飞蚊 17086.0
葩问题竞猜 17086.0
飞蚊症 17086.0
坛上有童鞋 17086.0
唤醒 17086.0
=zh。稀 17086.0
杯竟 17086.0
轨迹？。锐 17086.0
=zh。稀土 17086.0
堪称 17086.0
允许 17086.0
玛是复制粘贴 17086.0
枫叶主机优惠 17086.0
陈斌 17086.0
再迷茫!。毫 17086.0
谷歌 17086.0
负责 17086.0
沈阳 1